## **Model Tuning: Transfer Learning with Efficient Net**

In [1]:
# import necessary libraries
import os
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import tensorflow.keras.models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications import EfficientNetB0

## **Load image dataframe**

In [2]:
# choose day or night dataframe for reference
#infile = 'day_df.pkl'
#infile = 'night_df.pkl'
path = '/home/ubuntu/michael/my_pickles/'

In [4]:
# set reference dataframe
sample_df = pd.read_pickle(path + infile)

## **Set source directories**

In [13]:
# to access day images only
train_dir = '/home/ubuntu/michael/day/train'
val_dir = '/home/ubuntu/michael/day/validate'

In [5]:
# to access night images only
#train_dir = '/home/ubuntu/michael/night/train'
#val_dir = '/home/ubuntu/michael/night/validate'

## **Loading images from directories**

In [6]:
# clear memory
K.clear_session()

In [7]:
# set input size of images based on model specs
img_size = (224, 224)
img_shape = (224, 224, 3)

In [10]:
# create training data generator and augment training images
train_datagen = ImageDataGenerator(rescale = 1./255., rotation_range = 40, 
                                   width_shift_range = 0.2, height_shift_range = 0.2,
                                   shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [11]:
# create validation data generator and rescale validation images
valid_datagen = ImageDataGenerator(rescale = 1.0 / 255.)

In [14]:
# prepare trainging iterator
train_it = train_datagen.flow_from_directory(directory = train_dir, classes = ['nofog', 'fog'], class_mode = 'binary', # train_iterator
                    batch_size = 64, target_size=img_size) # batch size is number of steps

Found 897 images belonging to 2 classes.


In [16]:
class_names = train_it.classes
class_indices = train_it.class_indices

In [17]:
# prepare validation iterator
valid_it = valid_datagen.flow_from_directory(directory = val_dir, classes = ['nofog', 'fog'], class_mode = 'binary', # validation_iterator
                    batch_size = 16, target_size=img_size)

Found 112 images belonging to 2 classes.


In [18]:
class_names = valid_it.classes
class_indices = valid_it.class_indices

## **Load Efficient net model as convoutional base**

In [19]:
# load pretrained efficient net model and weights for convolution
conv_base = EfficientNetB0(weights="imagenet", include_top=False, input_shape = img_shape)

In [20]:
# set to false so weights are retained
conv_base.trainable = False

## **Add classifier on top of convolutional base**

In [21]:
# create classification portion of model
model = Sequential()
model.add(conv_base)
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

In [22]:
# view classification architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 11520)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              11797504  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 15,848,100
Trainable params: 11,798,529
Non-trainable params: 4,049,571
____________________________________

In [23]:
# compile model
model.compile(loss = 'binary_crossentropy', optimizer =  RMSprop(lr = 1e-6, decay=1e-6), metrics = ['acc'])

In [24]:
# set early stopping
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 0, 
                                      patience=15, restore_best_weights = True)

In [25]:
# define checkpoint path
outputFolder = '/home/ubuntu/michael/model_output/EfficientNet/'
#outputFolder = '/home/ubuntu/michael/model_output/EfficientNet/day/'
#outputFolder = '/home/ubuntu/michael/model_output/EfficientNet/night/'
if not os.path.exists(outputFolder):
    os.makedirs(outputFolder)

In [27]:
# define chekpoint path and add checkpoints
checkpoint_path = outputFolder + 'EfficientNet_model.hdf5'

# add checkpoints
cp = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', mode = 'min', save_best_only=True)

In [34]:
# fit model with train itertor and use valid_it as a validation dataset during training
history = model.fit(train_it, steps_per_epoch = len(train_it), # @ of steps is the number of batches per epoch
                    validation_data = valid_it, validation_steps = len(valid_it), 
                    epochs = 3, verbose = 1, callbacks=[es, cp]) 

In [33]:
# evaluate model
score = model.evaluate(valid_it, steps=len(valid_it), verbose = 1)

In [30]:
# save model validation score
with open(outputFolder + 'EfficientNet_score.pkl', 'wb') as f:pickle.dump(score, f)

In [31]:
# access training and validation loss and accuracy data
loss_values = history.history['loss']
acc_values = history.history['acc']
valLoss_values = history.history['val_loss']
valAccuracy_values = history.history['val_acc']

# create dataframe to store accuracy and loss data
history_df = pd.DataFrame()
history_df['Training Loss'] = history.history['loss']
history_df['Training Accuracy'] = history.history['acc']
history_df['Validation Loss'] = history.history['val_loss']
history_df['Validation Accuracy'] = history.history['val_acc']

In [32]:
history_df.to_pickle(outputFolder + 'EfficientNet_history.pkl')